# Collective.jl: A tool for finding patterns

Collective.jl is a Julia package designed to identify interesting *features* from a collection of words. This is a common mechanic in puzzles you might find at, for example, the MIT Mystery Hunt. 

This is a Jupyter notebook using the Julia programming language to run Collective.jl. You can learn more about Jupyter notebooks at <http://jupyter.org/> and about Julia at [julialang.org](http://docs.julialang.org/en/stable/manual/introduction/). 

If you want to use this package without installing Julia, you can use [juliabox](https://juliabox.com/). Just sign in at [juliabox.com](https://juliabox.com/) and upload this file. 

## Features

A *feature* is just a boolean property of a word, like `"contains the letter 'a'"` or `"is a palindrome"`. Many features can also include scalar quantities. For example, we might want to compute a family of features for a word's Scrabble score. Those features would look like: `"scrabble score == 1"`, `"scrabble score == 2"`, ..., `"scrabble score == n".` 

## Interestingness

A particular set of words can satisfy many features. For example, if I give you the list of words: `["questionable", "businesswoman", "exhaustion", "discouraged", "communicated", "hallucinogen", "sequoia"]`, you might tell me that they all contain the letter 'a'. That's true, but it's much more interesting to notice that they all contain *all 5 vowels*. We measure this degree of interestingness using the standard binomial probability distribution: given `n` words, we can compute the probability that `k` of them satisfy some feature `F` as:

	(n choose k) * f^k * (1 - f)^(n - k)

where `f` is the frequency with wich feature `F` occurs in the population of words (in this case, the dictionary). The most interesting feature is the one whose probability of `k` occurrences is smallest. 

# Installation

In [11]:
# If you don't have Collective.jl already, this will install it
# directly from github (even on JuliaBox):
isdir(Pkg.dir("Collective")) || Pkg.clone("https://github.com/rdeits/Collective.jl.git")

true

# Usage

In [12]:
# Load the package
using Collective

To determine the frequency distribution of each feature, we'll need a population of English words:

In [16]:
const words = wordlist(open(joinpath(Pkg.dir("Collective"), 
                                     "data", 
                                     "113809of.fic")));

Constructing a `Corpus` does all the hard work of compiling functions for every feature and testing their frequencies (this will take a few seconds):

In [17]:
corpus = Corpus(words)

Corpus with 752 features

Now we can use that corpus to solve puzzles. Here's a set of words:

In [18]:
puzzle = ["questionable", "businesswoman", "exhaustion", "discouraged", "communicated", "hallucinogen", "sequoia"]

7-element Array{String,1}:
 "questionable" 
 "businesswoman"
 "exhaustion"   
 "discouraged"  
 "communicated" 
 "hallucinogen" 
 "sequoia"      

We can run the feature statistics with `analyze()`:

In [20]:
results = analyze(corpus, puzzle)
results[1:5]

5-element Array{Collective.FeatureResult,1}:
 Feature: 0/7 matches, p=0.9999384950620512: has scrabble score 1
 Feature: 0/7 matches, p=0.9979106502875219: has scrabble score 2
 Feature: 0/7 matches, p=0.9905663542684028: has scrabble score 3
 Feature: 0/7 matches, p=0.9668969109147115: has scrabble score 4
 Feature: 0/7 matches, p=0.926295207128359: has scrabble score 5 

Analyze returns a vector of `FeatureResult`s, each of which contains:

* A human-readable description of the feature
* A function to evaluate that feature
* The number of words in the puzzle which match that feature
* The binomial probability of that number of matches

To get the best features, we can just sort that list:

In [22]:
sort(results)[1:5]

5-element Array{Collective.FeatureResult,1}:
 Feature: 7/7 matches, p=6.201497887698415e-15: has 5 unique vowels    
 Feature: 5/7 matches, p=3.530553756449584e-7: has 10 unique letters   
 Feature: 7/7 matches, p=6.767335359488334e-5: contains 'u'            
 Feature: 3/7 matches, p=0.0009855954016791447: contains 'u' at index 5
 Feature: 7/7 matches, p=0.0014321567657974046: contains 'o'           

That first feature has a probability of about $1$ in $6 \times 10^{15}$. It's extremely unlikely that 7 randomly chosen words would have 5 unique vowels each. So that's the feature we should use for whatever the next step of the puzzle is! 

If we don't want the whole sorted list of features, we can just get the single best one:

In [24]:
best_feature = minimum(results)

Feature: 7/7 matches, p=6.201497887698415e-15: has 5 unique vowels

## More examples

There are a variety of puzzles used for testing Collective, and they show off some of the interesting patterns it can detect. The full list can be found here: <https://github.com/rdeits/Collective.jl/tree/master/test/puzzles>

Some of those examples are reproduced below:

# Clockwork Orange (2013)
<http://web.mit.edu/puzzle/www/2013/coinheist.com/rubik/clockwork_orange/index.html>

In [40]:
minimum(analyze(corpus, ["armoredrecon", "hypapante", "commemorativebats", "derricktruck", "brownrot", "attorneysgeneral", "sacrosanct", "impromptu"]))

Feature: 8/8 matches, p=3.480975449342696e-5: has at least 2 letters repeated exactly 2 times each

# Venntersections (2014)
<http://www.mit.edu/~puzzle/2014/puzzle/venntersections/>

In [41]:
minimum(analyze(corpus, ["grimaced", "formally", "questionable", "discouraged", "communicated", "chrysalis", "saccharin"]))

Feature: 7/7 matches, p=6.454937703778084e-8: contains 'a' at index 4 from end

In [42]:
minimum(analyze(corpus, ["thumbtacks", "monologue", "testimony", "camel", "meteorology", "trampoline", "achievement"]))

Feature: 7/7 matches, p=1.1709177354405793e-5: contains 'm'

In [43]:
minimum(analyze(corpus, ["philharmonic", "mischievous", "alphabet", "restaurant", "leeching", "mushroom", "pioneer"]))

Feature: 7/7 matches, p=2.1365907082135037e-9: contains a greek letter

## Clustering

Collective also knows how to cluster words. Specifically, given a list of M words, and a number N < M, it will try to find the group of N words which are related by a single very interesting feature. For example:


In [26]:
puzzle = shuffle(["hugoweaving",
                          "mountaindew",
                          "mozambique",
                          "sequoia",
                          "annotation",
                          "artificial",
                          "individual",
                          "omnivorous",
                          "onlocation",
                          "almost",
                          "biopsy",
                          "chimp",
                          "films",
                          "ghost",
                          "tux",
                          "balked",
                          "highnoon",
                          "posted"]);

In [28]:
# Find the best cluster of size 6 from the list in puzzle, using
# statistics from corpus
cluster = best_cluster(corpus, puzzle, 6)

(String["films","biopsy","tux","ghost","chimp","almost"],Feature: 6/6 matches, p=2.0626137770374948e-14: has 0 reverse alphabetical bigrams)

The 6 words which were identified are: `["biopsy","films","chimp","almost","tux","ghost"]`. The feature which was returned says that all 6 words have no reverse alphabetical bigrams, and that the probability of such an occurrence is 1 in 2e-14. Putting it another way, it says that *every* word has only alphabetical bigrams, or, more plainly, every word has all of its letters in alphabetical order. 